<a href="https://colab.research.google.com/github/RajeshThevar/Funny-caption-generation/blob/master/zero_shot_llama2_testMappu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install -q kaggle transformers datasets accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00


# Load the data

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
train.head()

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


## Zero-shot Llama 2

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

# load base LLM model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-hf',
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Looking at logits for next token

In [ ]:
# Now let's apply this to multiple-choice

# First up, a way to get the prompt:
def question_from_row(row):
    prompt = f"""
    ### INSTRUCTION: ANSWER WITH A, B, C, D or E only.
    ### QUESTION: {row['prompt']}
    ### A: {row['A']}
    ### B: {row['B']}
    ### C: {row['C']}
    ### D: {row['D']}
    ### E: {row['E']}
    ### ANSWER:"""
    return prompt

# Pick a random row and create the prompt
row = train.sample(1).iloc[0]
prompt = question_from_row(row)
print("Prompt:\n", prompt)

# Now we pass it through the model and see which of the possible answer tokens is most likely
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
with torch.no_grad():
  final_logits = model(input_ids).logits[0, -1] # The last set of logits
answer_idxs = torch.tensor([tokenizer.encode(l)[-1] for l in 'ABCDE'])
answer_logits = final_logits[answer_idxs]
answer_probs = final_logits.softmax(dim=0)[answer_idxs]
print("Answer probs (all):", answer_probs)
print("Answer probs (only answer):", answer_logits.softmax(dim=0))
with torch.no_grad():
    top_k_probs, top_k_idxs = torch.topk(answer_logits, 3, dim=0)
print("Top 3", ['ABCDE'[k] for k in top_k_idxs])
print('Predicted answer:', 'ABCDE'[answer_probs.argmax()])
print('Actual answer', row['answer'])

Prompt:
 
    ### INSTRUCTION: ANSWER WITH A, B, C, D or E only.
    ### QUESTION: What is the Einstein@Home project?
    ### A: The Einstein@Home project is a project that aims to detect signals from supernovae or binary black holes. It takes data from LIGO and GEO and sends it out in little pieces to thousands of volunteers for parallel analysis on their home computers.
    ### B: The Einstein@Home project is a project that aims to detect signals from supernovae or binary black holes. It takes data from SETI and GEO and sends it out in little pieces to thousands of volunteers for parallel analysis on their home computers.
    ### C: The Einstein@Home project is a distributed computing project that aims to detect simple gravitational waves with constant frequency. It takes data from LIGO and GEO and sends it out in little pieces to thousands of volunteers for parallel analysis on their home computers.
    ### D: The Einstein@Home project is a project that aims to detect simple gravita

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Answer probs (all): tensor([0.2369, 0.1591, 0.1027, 0.1506, 0.1191])
Answer probs (only answer): tensor([0.3083, 0.2070, 0.1337, 0.1960, 0.1550])
Top 3 ['A', 'B', 'D']
Predicted answer: A
Actual answer C


In [ ]:
# See how accurate this is
from tqdm.auto import tqdm

def question_from_row(row):
    prompt = f"""
    ### INSTRUCTION: ANSWER WITH A, B, C, D or E only.
    ### QUESTION: {row['prompt']}
    ### A: {row['A']}
    ### B: {row['B']}
    ### C: {row['C']}
    ### D: {row['D']}
    ### E: {row['E']}
    ### ANSWER: """
    return prompt

first = 0
second = 0
third = 0
total = 0

for i, row in tqdm(train.iterrows(), total=len(train)):
    prompt = question_from_row(row)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    with torch.no_grad():
      final_logits = model(input_ids).logits[0, -1] # The last set of logits
    answer_idxs = answer_idxs = torch.tensor([tokenizer.encode(l)[-1] for l in 'ABCDE'])
    answer_logits = final_logits[answer_idxs]
    answer = row['answer']
    answer_probs = answer_logits.softmax(dim=0)


    with torch.no_grad():
        top_k_probs, top_k_idxs = torch.topk(answer_logits, 3, dim=0)

    if i < 5: # Some debugging
      print("Prompt:\n", prompt)
      print("Answer probs:", answer_probs)
      print("Top 3", ['ABCDE'[k] for k in top_k_idxs])
      print('Predicted answer:', 'ABCDE'[answer_probs.argmax()])
      print('Actual answer', answer)
    if 'ABCDE'[top_k_idxs[0]] == answer:
        first += 1
    if 'ABCDE'[top_k_idxs[1]] == answer:
        second += 1
    if 'ABCDE'[top_k_idxs[2]] == answer:
        third += 1
    total += 1

print("\n\nAccuracy @1:", first/total)
print("MAP@3: ", (first + second/2 + third/3) /total )

  0%|          | 0/200 [00:00<?, ?it/s]

Prompt:
 
    ### INSTRUCTION: ANSWER WITH A, B, C, D or E only.
    ### QUESTION: Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?
    ### A: MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."
    ### B: MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.
    ### C: MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.
    ### D: MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersi

## Perplexity

In [ ]:
input_ids = tokenizer("This is a very likely string", return_tensors="pt").input_ids
with torch.no_grad():
  final_logits = model(input_ids).logits
final_logits.shape # One logit per token (32000) for each token in the string

torch.Size([1, 7, 32000])

In [ ]:
for token_logits, token in zip(final_logits[0], input_ids[0][1:]): # Note the shift by 1
  print(token_logits.softmax(dim=0)[token]) # The probability for that particular token

tensor(0.0002)
tensor(0.1509)
tensor(0.4028)
tensor(0.0281)
tensor(5.6604e-05)
tensor(6.2966e-05)


In [ ]:
def calculate_perplexity(s):
  input_ids = tokenizer(s, return_tensors="pt").input_ids
  with torch.no_grad():
    final_logits = model(input_ids).logits
  perplexity = []
  for token_logits, token in zip(final_logits[0], input_ids[0][1:]):
    perplexity.append(-torch.log(token_logits.softmax(dim=0)[token])) # Cross entropy: -log(p(correct_answer))
  perplexity = torch.stack(perplexity, dim=0)
  perplexity = torch.mean(perplexity)
  return perplexity.cpu().item()

calculate_perplexity("This is a very likely string"),\
calculate_perplexity("This do string likely a")

(5.739597797393799, 10.101953506469727)

In [ ]:
row = train.sample(1).iloc[0]
perplexities = []

for possible_answer in 'ABCDE':
    prompt = f"""
    ### QUESTION: {row['prompt']}
    ### ANSWER: {row[possible_answer]}"""
    perplexity = calculate_perplexity(prompt)
    perplexities.append(perplexity)
    print(f"Perplexity for {possible_answer}: {perplexity}")

print("Correct answer:", row['answer'])
print("Perplexities:", perplexities)
print("Predicted answer:", 'ABCDE'[perplexities.index(min(perplexities))])

Perplexity for A: 1.8200336694717407
Perplexity for B: 1.7376810312271118
Perplexity for C: 1.700491189956665
Perplexity for D: 1.668608546257019
Perplexity for E: 1.740678310394287
Correct answer: D
Perplexities: [1.8200336694717407, 1.7376810312271118, 1.700491189956665, 1.668608546257019, 1.740678310394287]
Predicted answer: D


In [ ]:
# Apply this to all questions, scoring the same way we did with the previous approach

first = 0
second = 0
third = 0
total = 0

for i, row in tqdm(train.iterrows(), total=len(train)):
    perplexities = []

    for possible_answer in 'ABCDE':
        prompt = f"""### QUESTION: {row['prompt']}
                     ### ANSWER: {row[possible_answer]}"""
        perplexity = calculate_perplexity(prompt)
        perplexities.append(perplexity)

    top_k_probs, top_k_idxs = torch.topk(-torch.tensor(perplexities), 3, dim=0) # - so we get the three lowest perplexities

    if 'ABCDE'[top_k_idxs[0]] == row['answer']:
        first += 1
    if 'ABCDE'[top_k_idxs[1]] == row['answer']:
        second += 1
    if 'ABCDE'[top_k_idxs[2]] == row['answer']:
        third += 1
    total += 1

print("\n\nAccuracy @1:", first/total)
print("MAP@3: ", (first + second/2 + third/3) /total )

  0%|          | 0/200 [00:00<?, ?it/s]



Accuracy @1: 0.51
MAP@3:  0.635
